# LandVote Analysis
## Visualizations + Statistical Tests 

For details on data processing, please refer to the code for 
- [<u>LandVote</u>](https://github.com/cassiebuhler/datasets/tree/main/landvote)
- [<u>Census boundaires</u>](https://github.com/cassiebuhler/datasets/tree/main/census)
- [<u>Political party</u>](https://github.com/cassiebuhler/datasets/tree/main/political_parties)

In this script, we are visualizing the resulting LandVote data and running statistical tests.

In [1]:
import ibis
from ibis import _
import ibis.expr.datatypes as dt  
import re
from cng.utils import *
from cng.h3 import *
from minio import Minio
import altair as alt
from utils import *

duckdb_install_h3()
con = ibis.duckdb.connect(extensions = ["spatial"])
set_secrets(con)


# Load data 

In [2]:
votes= (con.read_parquet('s3://public-tpl/landvote/landvote_party.parquet')
    .drop('geom')
)

#  collapse multi-county measures to one row per landvote_id 
votes = get_unique_rows(votes)


### Sensitivity analysis


In [3]:
## uncomment for sensitivity analysis!
## reclassify parties if we do sensitivity analysis
# party_val = "Democrat" 
# votes = votes.mutate(party=_.party.substitute({'Mixed':party_val,"None":party_val,"Other":party_val,None:party_val}))

### Overall Pass Rate

In [4]:
get_passed(votes)
# compute percentage passed over entire dataset

77.25% Measures Passed from 1988 - 2024 



# Pass rates by jurisdiction party 

In [5]:
df = (
    votes
    .filter(_.party.isin(["Democrat", "Republican"]))
    .group_by("year", "party")
    .agg(pass_fraction=((_.status.isin(["Pass", "Pass*"]))).cast("int").mean())
    .order_by("year")
    .execute()
)

chart = year_line_mechanism(
    df,
    y="pass_fraction",
    group="party",
    title="Conservation Measure Pass Rates by Jurisdiction Party (1988–2025)",
    y_title="% Passed",
    stat='percent'
)


chart.save('percent_passed_party.png', ppi=200)
chart

alt.Chart(...)

## Equivalence Test - two one-sided t-tests (TOST)

Let $\Delta  = p_D-p_R$ where $\delta$ is a equivalence margin.

$H_{0}: \Delta \leq - \delta$ or $\Delta \geq \delta\\$ 
$H_{A}: -\delta < \Delta < \delta\\$

Short Conclusion:
Since both p-values $<0.05$ -> reject non-equivalence
Pass rates are statistically equivalent within $\pm5$ percentage points

Detailed:
The estimated difference in conservation measures passage rates between Democratic and Republican jurisdictions is 1.6 percentage points ($95% CI: −1.6$ to $4.8$). Using a two one-sided tests (TOST) equivalence framework with a $\pm5$ percentage point margin, we reject non-equivalence ($p < 0.05$ for both bounds), indicating statistically equivalent passage rates across parties.

In [6]:
import numpy as np
from statsmodels.stats.proportion import test_proportions_2indep, confint_proportions_2indep

df = votes.execute()  # one row per landvote_id
df = df[df["party"].isin(["Democrat", "Republican"])].copy()
df["passed"] = df["status"].isin(["Pass", "Pass*"]).astype(int)

g = df.groupby("party")["passed"].agg(["sum", "count"])
xD, nD = int(g.loc["Democrat", "sum"]), int(g.loc["Democrat", "count"])
xR, nR = int(g.loc["Republican", "sum"]), int(g.loc["Republican", "count"])

delta = 0.05   # equivalence margin: ±5 percentage points
alpha = 0.05

# TOST:
# 1) H0: (pD - pR) <= -delta  vs  H1: (pD - pR) > -delta
p_lo = test_proportions_2indep(xD, nD, xR, nR, value=-delta, alternative="larger").pvalue

# 2) H0: (pD - pR) >=  delta  vs  H1: (pD - pR) <  delta
p_hi = test_proportions_2indep(xD, nD, xR, nR, value= delta, alternative="smaller").pvalue

equivalent = (p_lo < alpha) and (p_hi < alpha)

# effect size + CI (helpful to report)
diff = xD / nD - xR / nR
ci_lo, ci_hi = confint_proportions_2indep(xD, nD, xR, nR, method="wald")

print("diff (pD - pR) =", diff)
print("95% CI =", (ci_lo, ci_hi))
print("TOST p-values =", (p_lo, p_hi))
print("Equivalent within ±delta?", equivalent)


diff (pD - pR) = 0.016303702009828003
95% CI = (-0.015658286938978437, 0.04826569095863444)
TOST p-values = (2.2524453542396927e-05, 0.020017487126888136)
Equivalent within ±delta? True


# Outcome by Finance Measure

In [7]:
df = (
    votes
    .mutate(
        other_comment_lc = _.other_comment.fill_null("").lower(),
        fm_lc = _.finance_mechanism.fill_null("").lower(),
    )
    .mutate(
        mechanism_group = ibis.ifelse(
            _.fm_lc.contains("bond"),
            "Bond",
            ibis.ifelse(
                # converting other to tax based on the other_comment column  
                (_.fm_lc.contains("tax")) | ((_.finance_mechanism == "Other") & (_.other_comment_lc.contains("tax"))),
                "Tax",
                "Other",
            ),
        ),
    )
    .filter(_.mechanism_group!="Other")
    .filter(_.party.isin(["Democrat", "Republican"]))
    .group_by("mechanism_group", "party")
    .agg(pass_fraction=(_.status.isin(["Pass", "Pass*"])).cast("int").mean())
    .execute()
)


chart = bar_chart(
    df,
    y="pass_fraction",
    group="party",
    title="Conservation Measure Pass Rates by Jurisdiction Party and Finance Mechanism",
    y_title="% Passed",
    stat='percent'
)


# chart.save('party_mechanism.png', ppi=200)
chart

alt.Chart(...)